In [101]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
import collections
from tensorflow.contrib import rnn
import h5py
import random
import pickle

from utils import normalize_adj, StandardScaler, masked_mae_tf, masked_mae_tf_by_horizon
from gcn import gcn, gcnn_ddgf

In [102]:
file_Name = "data/NYCBikeHourly272.pickle"
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)

In [103]:
node_num = 272 # node number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand
lstm_steps = 1 # number of cells in the LSTM part

X_whole = []
Y_whole = []

x_offsets = np.sort(
    np.concatenate((np.arange(-feature_in+1, 1, 1),))
)
# Predict the next one hour
y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(hourly_bike.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = hourly_bike.iloc[t + x_offsets, 0:sn].values.flatten('F')
    y_t = hourly_bike.iloc[t + y_offsets, 0:sn].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
Y_whole = np.stack(Y_whole, axis=0)

X_whole = np.reshape(X_whole, [X_whole.shape[0], node_num, feature_in])

i = lstm_steps
X_whole_lstm = []
Y_whole_lstm = []

while i <= X_whole.shape[0]:
    X_whole_lstm.append(X_whole[i-lstm_steps:i,:]) # features from i-lstm_steps + 1 to i
    Y_whole_lstm.append(Y_whole[i-1])
    i = i + 1

X_whole_lstm = np.stack(X_whole_lstm, axis = 0) # (34239, 10, 207, 12)
Y_whole_lstm = np.stack(Y_whole_lstm, axis = 0) # (34239, 2484)

num_train = 20000 # Note here actually we use the first 20000 to train the model. The paper mentioned "22304" need to be corrected.
num_val = 2000
num_test = 2000

X_training = X_whole_lstm[:num_train, :]
Y_training = Y_whole_lstm[:num_train, :]

# shuffle the training dataset
perm = np.arange(X_training.shape[0])
np.random.shuffle(perm)
X_training = X_training[perm]
Y_training = Y_training[perm]

X_val = X_whole_lstm[num_train:num_train+num_val, :]
Y_val = Y_whole_lstm[num_train:num_train+num_val, :]

X_test = X_whole_lstm[num_train+num_val:num_train+num_val+num_test, :]
Y_test = Y_whole_lstm[num_train+num_val:num_train+num_val+num_test, :]

scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

X_training = scaler.transform(X_training)
Y_training = scaler.transform(Y_training)

X_val = scaler.transform(X_val)
Y_val = scaler.transform(Y_val)

X_test = scaler.transform(X_test)
Y_test = scaler.transform(Y_test)

In [104]:
# Create model
def gcn(x, weights, biases, batch_size, n_input, frequency,flag, n_output_vec):
    # Hidden layer with RELU activation
    
   # output_list = tf.Variable(tf.zeros([n_output_vec,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre

    #Xtem = tf.reshape(x[i,:], [n_input, frequency])
    #Xtem = tf.transpose(Xtem)
    #Atem = tf.convert_to_tensor(A_whole_final, dtype=np.float32)
    #Atem1 = tf.diag(tf.ones([n_input])) # Atem not palying any roles
    #Ytem = tf.reshape(Y[i,:], [n_input, 1])
    #Atem = tf.diag(tf.ones([n_input]))
    #x = tf.reshape(x, [-1, sn, 1]) # 100, 207, 1
    
    # x (?, 207, 12)
    x = tf.transpose(x, [1, 0, 2]) # 207, ?, 12
    x = tf.reshape(x, [sn, -1]) # 207, batch*feature_num
    Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
    Atem1 = normalize_adj(Atem1)
    #th = tf.constant(0.01, dtype=tf.float32)
    #where = tf.subtract(Atem1, th)
    #Atem1 = tf.nn.relu(where)

    Z1 = tf.matmul(Atem1, x) # 207, batch*feature_num  #+ tf.matmul( tf.matmul(weights['A1'], weights['A1']), Xtem)
    Z1 = tf.reshape(Z1, [-1, frequency]) # 207* 100, frequency
    #layer_1 = tf.matmul(Z1, weights['h1']) 
    layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1) # 207*100, hidden1

    
    Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
    Atem2 = normalize_adj(Atem2)
    
    layer_1 = tf.reshape(layer_1, [sn, -1])  # 207, batchsize*hidden1
    Z2 = tf.matmul(Atem2, layer_1)
    Z2 = tf.reshape(Z2, [-1, n_hidden_vec1]) # 207*batchsize, n_hidden_vec1
    layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2) # 207*batchsize, hidden2

    Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
    Atem3 = normalize_adj(Atem3) 
    
    layer_2 = tf.reshape(layer_2, [sn, -1])  # 207, batchsize*hidden2
    Z3 = tf.matmul(Atem3, layer_2)
    Z3 = tf.reshape(Z3, [-1, n_hidden_vec2]) # 207*batchsize, hidden2
    layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3) # 207*batchsize, hidden3
    
    #Atem4 = 0.5*(weights['A4'] + tf.transpose(weights['A4']))#+ Atem 
    #Atem4 = normalize_adj(Atem4)
    #Z4 = tf.matmul(Atem4, layer_3)
    #layer_4 = tf.add(tf.matmul(Z4, weights['h4']), biases['b4'])
    #layer_4 = tf.nn.relu(layer_4)

    # flattern
    #layer_3 = tf.reshape(layer_3, [1, 272*n_hidden_vec3])

    #F1 = tf.add(tf.matmul(layer_3, weights['f1']), biases['bf1'])
    #F1 = tf.nn.relu(F1)

    #F2 = tf.add(tf.matmul(F1, weights['f2']), biases['bf2'])
    #F2 = tf.nn.relu(F2)

    #F3 = tf.add(tf.matmul(F1, weights['f3']), biases['bf3'])
    #out_layer = tf.reshape(F3, [272, 1])

    # Output layer with linear activation
    layer_3 = tf.reshape(layer_3, [sn, -1, n_hidden_vec3])
    layer_3 = tf.transpose(layer_3, [1, 0, 2]) # batchsize, sn, hidden3
    layer_3 = tf.reshape(layer_3, [-1, sn*n_hidden_vec3]) # batchsize, sn*hidden3
    Z4 = layer_3
    #out_layer = tf.add(tf.matmul(Z4, weights['outg']), biases['boutg'])
    #tf.matmul(Atem1, layer_1)#tf.matmul(Atem, layer_3)
    #out_layer = tf.add(tf.matmul(Z4, weights['out']), biases['bout'])
    #out_layer = tf.nn.relu(out_layer)
    # weather layer 1
    #x_wea_tem = tf.reshape(x_wea[i,:], [1, 9*frequency2]) # 1 by 126
    #layer_1_wea = tf.add(tf.matmul(x_wea_tem, weights['h1_wea']), biases['b1_wea'])
    #layer_1_wea = tf.nn.relu(layer_1_wea)

    #out_layer_wea = tf.add(tf.matmul(layer_1_wea, weights['out_wea']), biases['bout_wea'])
    #out_layer = tf.add(out_layer, tf.reshape(out_layer_wea, [272, 1]))


    
    return Z4

In [105]:
def gcn_corr_final(frequency, horizon, learning_rate, decay,batch_size, n_hidden_vec1,n_hidden_vec2,n_hidden_vec3,keep, early_stop_th,training_epochs, reg1, reg2):
    # set size
    #sn = 3 # station number

    
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []
    Y_true = []

    tf.reset_default_graph()

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    
    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    # Network Parameters

    #n_classes = 2 # MNIST total classes (0-9 digits) # n_classes is for classification only

    tf.reset_default_graph()

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, lstm_steps, sn, feature_in]) # X is the input signal
    #X_weather = tf.placeholder(tf.float32, [None, 9 * frequency2]) # X_weather weather and holiday information (9 is the feature number)
    A = tf.placeholder(tf.float32, [None, n_A_vec]) # A is the normalized adj matrix
    oldA = tf.placeholder(tf.float32, [n_input, n_input])
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output
    #num = tf.placeholder(tf.int32,[1, 1] )

    #Xtem = tf.placeholder(tf.float32, [n_input, frequency]) # for each row of X, A, Y, it can be reshaped to Xtem, Atem, Ytem
    #Atem = tf.placeholder(tf.float32, [n_input, n_input]) # 
    #Ytem = tf.placeholder(tf.float32, [n_input, 1]) #

    #Ypre = tf.placeholder(tf.float32, [None, n_output_vec])

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([frequency, n_hidden_vec1]), dtype=np.float32),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2]), dtype=np.float32),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3]), dtype=np.float32),
        #'h4': tf.Variable(tf.random_normal([n_hidden_vec3, n_hidden_vec4])),
        'outg': tf.Variable(tf.random_normal([sn*n_hidden_vec3, n_hidden_vec4]), dtype=np.float32), 
        'out': tf.Variable(tf.random_normal([num_hidden, Y_whole.shape[1]]), dtype=np.float32), # dont forget to change n_hidden_vec1 when add/delete layers
        #'f1': tf.Variable(tf.random_normal([272*n_hidden_vec3, 100])),
        #'f2': tf.Variable(tf.random_normal([50, 10])),
        #'f3': tf.Variable(tf.random_normal([100, 272])),
        'A1': tf.Variable(tf.random_normal([n_input,n_input]), dtype=np.float32),
        'A2': tf.Variable(tf.random_normal([n_input,n_input]), dtype=np.float32),
        'A3': tf.Variable(tf.random_normal([n_input,n_input]), dtype=np.float32),
        #'A4': tf.Variable(tf.random_normal([n_input,n_input])),
        #'h1_wea': tf.Variable(tf.random_normal([9*frequency2, n_hidden_weather1])),
        #'out_wea': tf.Variable(tf.random_normal([n_hidden_weather1, n_input]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([1, n_hidden_vec1]), dtype=np.float32),# n_hidden_vec1])),# bias all the same??? dont forget to test
        'b2': tf.Variable(tf.random_normal([1, n_hidden_vec2]), dtype=np.float32), #n_hidden_vec2])),
        'b3': tf.Variable(tf.random_normal([1, n_hidden_vec3]), dtype=np.float32),#n_hidden_vec3])),
        #'b4': tf.Variable(tf.random_normal([n_input, n_hidden_vec4])),
        #'b1': tf.Variable(tf.random_normal([n_input,n_hidden_vec1])),# bias all the same??? dont forget to test
        #'b2': tf.Variable(tf.random_normal([n_input,n_hidden_vec2])),
        #'b3': tf.Variable(tf.random_normal([n_input,n_hidden_vec3])),
        #'bf1': tf.Variable(tf.random_normal([1, 100])), 
        #'bf2': tf.Variable(tf.random_normal([1, 10])), 
        #'bf3': tf.Variable(tf.random_normal([1, 272])), 
        'boutg': tf.Variable(tf.random_normal([1, n_hidden_vec4]), dtype=np.float32), 
        'bout': tf.Variable(tf.random_normal([Y_whole.shape[1]]), dtype=np.float32), 
        #'b1_wea': tf.Variable(tf.random_normal([1, n_hidden_weather1])), 
        #'bout_wea': tf.Variable(tf.random_normal([1, n_input])), 
    }
    
    with tf.variable_scope('lstm'):
        lstm = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)
        rnn_input_seq = tf.unstack(X, lstm_steps, 1) # lstm_steps is the 2nd variable

        for i in range(lstm_steps):
            rnn_input_seq[i] = gcn(rnn_input_seq[i], weights, biases, batch_size,n_input, frequency, 1, n_output_vec)
            #print (rnn_input_seq[i].shape)
        outputs, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq, dtype=tf.float32)
        output_reshape = tf.reshape(outputs[-1], [-1, num_hidden])
        #print ('123here!!!!!!!!!!!')
        pred = tf.matmul(output_reshape, weights['out']) + biases['bout']
        #print (pred)
        #pred = tf.reshape(pred, [-1, Y_whole.shape[1]])
        #print ('here!!!!!!!!!!!')
        pred = scaler.inverse_transform(pred)
        Y_true_tr = scaler.inverse_transform(Y)

        cost = tf.reduce_mean(tf.pow(pred - Y_true_tr, 2)) 
        #print (cost)
        
    with tf.variable_scope('lstm', reuse=True):
        rnn_input_seq_val = tf.unstack(X, lstm_steps, 1)

        for i in range(lstm_steps):
            rnn_input_seq_val[i] = gcn(rnn_input_seq_val[i], weights, biases, batch_size,n_input,frequency, 2, n_output_vec)
        outputs_val, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq_val, dtype=tf.float32)
        output_reshape = tf.reshape(outputs_val[-1], [-1, num_hidden])

        pred_val = tf.matmul(output_reshape, weights['out']) + biases['bout']
        pred_val = scaler.inverse_transform(pred_val)
        Y_true_val = scaler.inverse_transform(Y)
        
        cost_val =  tf.reduce_mean(tf.pow(pred_val - Y_true_val, 2)) 
        #print ('234here!!!!!!!!!!!')
    with tf.variable_scope('lstm', reuse=True):
        rnn_input_seq_test = tf.unstack(X, lstm_steps, 1)
        
        for i in range(lstm_steps):
            rnn_input_seq_test[i] = gcn(rnn_input_seq_test[i], weights, biases, batch_size,n_input,frequency, 3, n_output_vec)
        outputs_test, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq_test, dtype=tf.float32)
        output_reshape = tf.reshape(outputs_test[-1], [-1, num_hidden])

        pred_tes = tf.matmul(output_reshape, weights['out']) + biases['bout']
        pred_tes = scaler.inverse_transform(pred_tes)
        Y_true_tes = scaler.inverse_transform(Y)
        
        cost_tes = tf.reduce_mean(tf.pow(pred_tes - Y_true_tes, 2)) 
        
        #print ('345here!!!!!!!!!!!')
    #rmse
    #cost_tes = tf.reduce_mean(tf.pow(pred_tes-Y, 2))
    # cross-entropy for classification
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_train))
    # ratio = tf.abs(tf.reduce_sum(pred)-tf.reduce_sum(Y))/tf.reduce_sum(Y)
    #zero = 0
    #ratio = tf.reduce_mean(tf.divide(tf.where(tf.not_equal(Y, zero), np.abs(pred-Y), tf.zeros(Y.get_shape(), tf.float32)), tf.where(tf.not_equal(Y, zero), Y, tf.ones(Y.get_shape(), tf.float32))))
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    #total_val_cost = []
    #total_val_ratio = []

    # learning start from 

    #index = daily_bike[(daily_bike['year'] == 2016) & (daily_bike['monthofyear'] == 1) & (daily_bike['dayofmonth'] == 1)].index.tolist()[0]
    #A_hat = normalize_adj(corr_matrix_trips)
    #print(A_hat)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size) #int(num_train/batch_size)

            for i in range(total_batch):
                #print (Y_training[i*batch_size:(i+1)*batch_size,].size())
                #num = batch_size
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,],  
                                                              keep_prob: keep})
                #print (preds)
                #print (trueval)
                #print ("Epoch:", '%04d' % (epoch+1), "batch: ", i, "batch cost=", \
                #    "{:.9f}".format(c))
                #print ('here!!!!!!!!!!!!!!!!')
                avg_cost += c * batch_size #/ total_batch 
                #Display logs per epoch step
                
            # rest part of training dataset
            #num = num_train - total_batch*batch_size 
            if total_batch * batch_size != num_train:
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[total_batch*batch_size:num_train,], 
                                          Y: Y_training[total_batch*batch_size:num_train,],
                                                  keep_prob: keep})
                avg_cost += c * (num_train - total_batch*batch_size)
            
            avg_cost = np.sqrt(avg_cost / num_train)
            
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format(avg_cost)) #np.sqrt(avg_cost)
                
            # also use batch to save memory
            # validation
            c_val = 0.
            total_bat_val = int(num_val/batch_size)
            for i in range(total_bat_val):
                #num = batch_size
                c_val_b = sess.run([cost_val], feed_dict={X: X_val[i*batch_size:(i+1)*batch_size,], 
                                                          Y: Y_val[i*batch_size:(i+1)*batch_size,],   keep_prob:1})
                c_val += c_val_b[0]*batch_size
            
            if total_bat_val * batch_size != num_val:
                #num = num_val - total_bat_val*batch_size
                c_val_b = sess.run([cost_val], feed_dict={X: X_val[total_bat_val*batch_size:num_val,], 
                                                          Y: Y_val[total_bat_val*batch_size:num_val,],  keep_prob:1})
                c_val += c_val_b[0] * (num_val - total_bat_val*batch_size)
                
            c_val = np.sqrt(c_val / num_val)
            
            print("Validation RMSE: ", c_val)
            
            # test
            c_tes = 0.
            total_bat_test = int(num_test/batch_size)
            
            pre_test_tem = [] # save the prediction results
            Y_tes_true = []
            
            for i in range(total_bat_test):
                #num = batch_size
                c_tes_b, pred_tes1, Y_tes_batch = sess.run([cost_tes, pred_tes, Y_true_tes], feed_dict={X: X_test[i*batch_size:(i+1)*batch_size,],
                                                                               Y: Y_test[i*batch_size:(i+1)*batch_size,],  keep_prob: 1})
                c_tes += c_tes_b*batch_size

                #print (cost_h)
                #print (pred_tes1.shape)
                pre_test_tem.append(pred_tes1)
                Y_tes_true.append(Y_tes_batch)
                
            if total_bat_test * batch_size != num_test:
                #num = num_test - total_bat_test*batch_size
                c_tes_b, pred_tes1, Y_tes_batch = sess.run([cost_tes, pred_tes, Y_true_tes], feed_dict={X: X_test[total_bat_test*batch_size:num_test,],
                                                                               Y: Y_test[total_bat_test*batch_size:num_test,],  keep_prob: 1})
                c_tes += c_tes_b * (num_test - total_bat_test*batch_size) 
                 
                
                #print (pred_tes1.shape)
                pre_test_tem.append(pred_tes1)
                Y_tes_true.append(Y_tes_batch)
            
            #print (c_tes_h.shape)
            pre_test_tem = np.concatenate(pre_test_tem, axis = 0)
            #print (pre_test_tem.shape)
            Y_tes_true = np.concatenate(Y_tes_true, axis = 0)
            
            c_tes = np.sqrt(c_tes / num_test)
            #c_tes_h = c_tes_h / num_test
            
            print("Test RMSE: ", c_tes)
            #print("predic step: ", cost_by_hor)

            if c_val < best_val:
                best_val = c_val
                #saver.save(sess, './bikesharing_graph_2_th_point1')
                test_error = c_tes
                
                traing_error = avg_cost#np.sqrt(avg_cost)
                early_stop_k = 0 # reset to 0
                #print (pred_tes1)
                predic_res = pre_test_tem
                Y_true = Y_tes_true
                #predic_step = cost_by_hor

            # early stopping
            if c_val >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
              #  print ("early stopping...")
                break
            

        print("epoch is ", epoch)
        print("training RMSE is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", best_val)#(np.sqrt(best_val)))
        print("The test RMSE is ", test_error)#(np.sqrt(test_error)))
    
    #test_Y = Y_test
    #test_error = np.sqrt(test_error)
    return test_error, predic_res, Y_true#, A1#, predic_step#, A2

In [106]:

import datetime
#freq_max = 12
#time_step = 12
learning_rate = 0.005
decay = 0.9 
batch_size = 1000
num_hidden = 50 # number of hiddent units in LSTM Cell
early_stop_th = 150
training_epochs = 1000
keep = 1#0.2
#time_step_max = 10

sn = 272 # station num

n_hidden_vec1 = 10
n_hidden_vec2 = 10
n_hidden_vec3 = 10
#n_hidden_vec4 = 10
#num = 0
#All_pred = np.empty([2000, 207])
#All_Y = np.empty([2000, 207])

#24*90
#step = 0
#gap = 100
#training = 0.7
#validation = 0.1
#test = 0.2

#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)



rep = 1 # repeating times

#total_sn = 0
#num_iter = 50
#init_points = 200


# stdbscan
#spatial_threshold = 300
#temporal_threshold = 300
#min_neighbors = 1 # number of neighbor

#frequency2 = skip1 + freq_max + training

#while step < 2000:

#hourly_bike_cluster = hourly_bike
best = -10000
pre_best = []
test_Y_best = []
test_error_best = 1000
A1_best = []
# A2_best = []
for i in range(rep):
    a = datetime.datetime.now()
    test_error, predic_res, Y_true = gcn_corr_final(frequency, horizon, learning_rate, decay, batch_size, n_hidden_vec1,
                                                                n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs, reg1, reg2)
    #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
    #print ("finished A running: ", i)
    b = datetime.datetime.now()
    print(b-a)
    


    

Epoch: 0001 Training RMSE= 8.214547363
Validation RMSE:  7.9315295255085365
Test RMSE:  7.029860295607363
Epoch: 0002 Training RMSE= 7.511738251
Validation RMSE:  7.558921408636187
Test RMSE:  6.6482448527556794
Epoch: 0003 Training RMSE= 7.252211687
Validation RMSE:  7.293514286472102
Test RMSE:  6.358865403511125
Epoch: 0004 Training RMSE= 6.985178979
Validation RMSE:  7.0327229631204
Test RMSE:  6.074397103490884
Epoch: 0005 Training RMSE= 6.724559781
Validation RMSE:  6.7927983127159255
Test RMSE:  5.8112658759458435
Epoch: 0006 Training RMSE= 6.487968754
Validation RMSE:  6.575557252890066
Test RMSE:  5.571095715175908
Epoch: 0007 Training RMSE= 6.283824685
Validation RMSE:  6.451980304562613
Test RMSE:  5.37408049059416
Epoch: 0008 Training RMSE= 6.096662778
Validation RMSE:  6.234002353172115
Test RMSE:  5.173329220914353
Epoch: 0009 Training RMSE= 5.957270295
Validation RMSE:  6.124635023803842
Test RMSE:  5.039449135784891
Epoch: 0010 Training RMSE= 5.756134758
Validation RMSE

Epoch: 0079 Training RMSE= 3.353073178
Validation RMSE:  3.863914445569659
Test RMSE:  3.113817922555015
Epoch: 0080 Training RMSE= 3.342843255
Validation RMSE:  3.7160602106158236
Test RMSE:  2.949900365213732
Epoch: 0081 Training RMSE= 3.368715496
Validation RMSE:  3.815346765019847
Test RMSE:  3.0543751666695598
Epoch: 0082 Training RMSE= 3.337321082
Validation RMSE:  3.7081732483217515
Test RMSE:  2.9550733244781933
Epoch: 0083 Training RMSE= 3.315833922
Validation RMSE:  3.7129404545249116
Test RMSE:  2.9607145280852705
Epoch: 0084 Training RMSE= 3.321336928
Validation RMSE:  3.778222377333098
Test RMSE:  2.997538430903743
Epoch: 0085 Training RMSE= 3.312777180
Validation RMSE:  3.6924289800365186
Test RMSE:  2.986564112957508
Epoch: 0086 Training RMSE= 3.302854689
Validation RMSE:  3.7005208190159147
Test RMSE:  2.9753136605716346
Epoch: 0087 Training RMSE= 3.283417565
Validation RMSE:  3.7358225805970404
Test RMSE:  3.041561246813096
Epoch: 0088 Training RMSE= 3.289168860
Valida

Epoch: 0157 Training RMSE= 2.753169241
Validation RMSE:  3.1846035157513737
Test RMSE:  2.446085554783561
Epoch: 0158 Training RMSE= 2.757152684
Validation RMSE:  3.1809349518606553
Test RMSE:  2.448067962083316
Epoch: 0159 Training RMSE= 2.740671583
Validation RMSE:  3.2055351308094435
Test RMSE:  2.4583546481313006
Epoch: 0160 Training RMSE= 2.746513259
Validation RMSE:  3.2111235330342134
Test RMSE:  2.4618692238888333
Epoch: 0161 Training RMSE= 2.746782371
Validation RMSE:  3.2284068823730516
Test RMSE:  2.4865832800941927
Epoch: 0162 Training RMSE= 2.744401904
Validation RMSE:  3.1833822542862014
Test RMSE:  2.441421533155289
Epoch: 0163 Training RMSE= 2.744650723
Validation RMSE:  3.178240135005325
Test RMSE:  2.4433055112468156
Epoch: 0164 Training RMSE= 2.736630596
Validation RMSE:  3.2336077678530493
Test RMSE:  2.499534515914791
Epoch: 0165 Training RMSE= 2.735935546
Validation RMSE:  3.188999646028611
Test RMSE:  2.4453930826285384
Epoch: 0166 Training RMSE= 2.740480745
Vali

Epoch: 0235 Training RMSE= 2.659233863
Validation RMSE:  3.1273177135107
Test RMSE:  2.4037667378732053
Epoch: 0236 Training RMSE= 2.662679234
Validation RMSE:  3.124996757505642
Test RMSE:  2.4121406584035396
Epoch: 0237 Training RMSE= 2.653686835
Validation RMSE:  3.1689016377359183
Test RMSE:  2.481613880063192
Epoch: 0238 Training RMSE= 2.658660580
Validation RMSE:  3.1183040215932434
Test RMSE:  2.410980578252157
Epoch: 0239 Training RMSE= 2.657996131
Validation RMSE:  3.1321454943409535
Test RMSE:  2.4079987747848284
Epoch: 0240 Training RMSE= 2.659795540
Validation RMSE:  3.105268517522755
Test RMSE:  2.402752154611176
Epoch: 0241 Training RMSE= 2.650005320
Validation RMSE:  3.1430016629939543
Test RMSE:  2.4427248880331023
Epoch: 0242 Training RMSE= 2.658497943
Validation RMSE:  3.1149513809671547
Test RMSE:  2.406019125427183
Epoch: 0243 Training RMSE= 2.655083389
Validation RMSE:  3.120813731975311
Test RMSE:  2.4154870727957505
Epoch: 0244 Training RMSE= 2.648733351
Validati

Epoch: 0313 Training RMSE= 2.610423091
Validation RMSE:  3.0843088222156223
Test RMSE:  2.408587868207245
Epoch: 0314 Training RMSE= 2.608403442
Validation RMSE:  3.091002274832356
Test RMSE:  2.438758231911191
Epoch: 0315 Training RMSE= 2.614155649
Validation RMSE:  3.076813859101582
Test RMSE:  2.3959151433715387
Epoch: 0316 Training RMSE= 2.607696427
Validation RMSE:  3.0625416499828435
Test RMSE:  2.4029471779302796
Epoch: 0317 Training RMSE= 2.606729804
Validation RMSE:  3.1007913548465162
Test RMSE:  2.437706131655519
Epoch: 0318 Training RMSE= 2.612624095
Validation RMSE:  3.0732101472700246
Test RMSE:  2.4040333830472114
Epoch: 0319 Training RMSE= 2.601661612
Validation RMSE:  3.106733801024946
Test RMSE:  2.4613764780075087
Epoch: 0320 Training RMSE= 2.610328313
Validation RMSE:  3.082962610966449
Test RMSE:  2.417239619989584
Epoch: 0321 Training RMSE= 2.604255474
Validation RMSE:  3.0844394572749327
Test RMSE:  2.391929519865276
Epoch: 0322 Training RMSE= 2.605498718
Validat

Epoch: 0391 Training RMSE= 2.575677508
Validation RMSE:  3.068260769197532
Test RMSE:  2.416226007033578
Epoch: 0392 Training RMSE= 2.573025732
Validation RMSE:  3.096704247749301
Test RMSE:  2.436771479577611
Epoch: 0393 Training RMSE= 2.578396638
Validation RMSE:  3.0476264222657576
Test RMSE:  2.400680685869827
Epoch: 0394 Training RMSE= 2.572707686
Validation RMSE:  3.0628554468770406
Test RMSE:  2.407295247736509
Epoch: 0395 Training RMSE= 2.578369286
Validation RMSE:  3.061942653153877
Test RMSE:  2.4003533282071174
Epoch: 0396 Training RMSE= 2.572195042
Validation RMSE:  3.065344559871972
Test RMSE:  2.40839681596696
Epoch: 0397 Training RMSE= 2.578236368
Validation RMSE:  3.052538845368078
Test RMSE:  2.402449095362375
Epoch: 0398 Training RMSE= 2.572339075
Validation RMSE:  3.0561996190573826
Test RMSE:  2.3911105890708755
Epoch: 0399 Training RMSE= 2.569986674
Validation RMSE:  3.0505620704292156
Test RMSE:  2.4064935338028364
Epoch: 0400 Training RMSE= 2.574654438
Validation

Epoch: 0469 Training RMSE= 2.556291931
Validation RMSE:  3.0413218437768275
Test RMSE:  2.3844710284453017
Epoch: 0470 Training RMSE= 2.553503372
Validation RMSE:  3.0818254718811935
Test RMSE:  2.4293694978008373
Epoch: 0471 Training RMSE= 2.551151318
Validation RMSE:  3.0652351232581005
Test RMSE:  2.417819953740318
Epoch: 0472 Training RMSE= 2.557836000
Validation RMSE:  3.0564424590690358
Test RMSE:  2.3989399933427986
Epoch: 0473 Training RMSE= 2.546742142
Validation RMSE:  3.0682890924651796
Test RMSE:  2.429248046384275
Epoch: 0474 Training RMSE= 2.550932889
Validation RMSE:  3.07275774701065
Test RMSE:  2.40944335880108
Epoch: 0475 Training RMSE= 2.553385874
Validation RMSE:  3.09409704092195
Test RMSE:  2.4464141509155155
Epoch: 0476 Training RMSE= 2.551373063
Validation RMSE:  3.07900903080616
Test RMSE:  2.4263061840031863
Epoch: 0477 Training RMSE= 2.545677475
Validation RMSE:  3.088834923410772
Test RMSE:  2.435148865851037
Epoch: 0478 Training RMSE= 2.561279914
Validation

KeyboardInterrupt: 

In [ ]:
#lstm cell num 10, hidden num [10, 10, 5], batchsize 100, hidden state 50: 4.14, not decreasing
#lstm cell num 10, hidden num [10, 10, 5], batchsize 500, hidden state 50: 2.43
#lstm cell num 10, hidden num [10, 10, 10], batchsize 500, hidden state 50: 4.13, not dropping
#lstm cell num 10, hidden num [10, 10, 10], batchsize 1000, hidden state 50:4.15

In [ ]:
#np.savetxt("lstm_gcnn_prediction_1.95.csv", predic_res, delimiter = ',')
#np.savetxt("lstm_gcnn_prediction_1.95_Y.csv", Y_true, delimiter = ',')